In [11]:
import torch
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn
import torch.optim as optim
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Device: cpu


In [12]:
# Load pre-trained ResNet-50
model_selection = 'old'  # Change to 'new' to use pre-trained ImageNet weights
model_state_dict_path = 'resnet50_smile_model.pth'

if model_selection == 'new':
    model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
elif model_selection == 'old':
    model = models.resnet50(weights=None)
    # Replace last layer with a custom one of four classes
    num_classes = 4
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    if os.path.exists(model_state_dict_path):
        model.load_state_dict(torch.load(model_state_dict_path, map_location=device))
        print("Model loaded from:", model_state_dict_path)
    else:
        print("Model file not found. Please train the model first.")


Model loaded from: resnet50_smile_model.pth


C:\Users\Y SANDEEP\AppData\Local\Temp\ipykernel_6132\2011624431.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_state_dict_path, 

In [13]:
# Modify final layer for 4 classes
num_classes = 4  # real smile, fake smile, neutral, anxiety_depression
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layer4 and fc layers
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

model = model.to(device)


In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005)

In [15]:
# Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# Data paths
train_path = r"C:\Users\Y SANDEEP\Desktop\Emotion-Based Smile Classifier\train"
test_path = r"C:\Users\Y SANDEEP\Desktop\Emotion-Based Smile Classifier\test"

# Load datasets
full_dataset = datasets.ImageFolder(train_path, transform=transform_train)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test  # change transform for val

test_dataset = datasets.ImageFolder(test_path, transform=transform_test)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [17]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Accuracy: {val_acc:.2f}%")

    # Save model after each epoch
    torch.save(model.state_dict(), "resnet50_smile_model.pth")


Epoch 1/5, Loss: 0.2412, Val Accuracy: 93.69%
Epoch 2/5, Loss: 0.0819, Val Accuracy: 94.53%
Epoch 3/5, Loss: 0.0483, Val Accuracy: 94.06%
Epoch 4/5, Loss: 0.0379, Val Accuracy: 92.69%
Epoch 5/5, Loss: 0.0316, Val Accuracy: 93.27%


In [18]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 73.93%


In [20]:
# Save model to scripts
torch.save(model.state_dict(), "resnet50_smile_model.pth")
print("Model saved to resnet50_smile_model.pth")


Model saved to resnet50_smile_model.pth
